In [59]:
#Remain bugged with PUCT, not knowing what exactly is prior of taking that action, 1/validmoves?
#mcts with greedy should be able to know the solution?

from game import * #game imported env
from copy import deepcopy

In [60]:
c_puct = 0.25
N = 3
eps = 1e-5

In [61]:
class PUCTnode(object):
    #after state node with PUCT value, child, parent, recorded
    def __init__(self, parent, player, action, prior=0.):
        self.parent = parent
        self.player = player
        self.action = action
        #non zero value for stabilization
        self.N = 2 #visit count
        self.W = 1 #winning count
        self.P = prior
        self.children = list()
        self.cidx = [-1]*(N*N)
        
    def score(self, c = c_puct): #the PUCT score
        return (self.W/self.N) + c*self.P*np.sqrt(self.parent.N)/(1+self.N)
    
    def best_child(self):
        mchildren = None
        mscore = -1 #score>0
        for child in self.children:
            if child.score() - mscore > eps:
                mchildren = [child]
                mscore = child.score()
            elif abs(child.score() - mscore) < eps:
                mchildren.append(child)
        if mchildren == None:
            return None
        mc = np.random.choice(mchildren)
        return mc

In [62]:
class PUCT_Agent(object):
    def __init__(self):
        pass
    def gen_move(self, env, simulation_cnt = 900):
        tree = PUCT(env)
        for ep in range(simulation_cnt):
            tree.run_cycle()
            #print("cycle\n", ep)
        p = tree.getPi(tree.root)
        return np.argmax(p)
        

class PUCT(object):
    def __init__(self, env, N=N):
        self.n = N
        self.n2 = N*N
        self.renv = deepcopy(env)
        self.simu_env = deepcopy(env)
        self.root = PUCTnode(None, -self.renv.player_to_move(), N*N) #
        self.expand(self.root) #watch out hirozen effect
        self.model = Random_Agent()
    
    def reset(self):
        self.simu_env = deepcopy(self.renv)
    
    def clear(self):
        self.renv = deepcopy(env)
        self.simu_env = deepcopy(self.renv)
        self.root = PUCTnode(None, -1, N*N)
    
    def expand(self, node):
        if len(node.children) != 0:
            raise AlgoError("want to expand already expanded node")
            return 
        for action in self.simu_env.valid_actions():
            node.cidx[action] = len(node.children)
            node.children.append(PUCTnode(node, self.simu_env.player_to_move(), action, 1./len(self.simu_env.valid_actions())))
        return len(node.children) != 0
    
    def simulate(self):
        env = self.simu_env
        while env.status == -2:
            pos = self.model.gen_move(env) #forget to add color in attributes
            env.take_action(pos)
        return env.status()
    
    def run_cycle(self):
        #reset
        self.reset()
        env = self.simu_env
        #selection 
        cur = self.root
        while cur.best_child() != None:
            cur = cur.best_child()
            env.take_action(cur.action)
        #expansion
        if(self.expand(cur)): #if not end
            pos = self.model.gen_move(env) #one step look ahead with greedy
            cur = cur.children[cur.cidx[pos]]
            env.take_action(pos)
            
        #    env.take_action(cur.action)
        #simulation
        winner = self.simulate()
        #backpropagation
        while True:
            cur.N += 1
            if cur.player == winner:
                cur.W += 1
            elif cur.player == -winner:
                cur.W -= 1
            if cur.parent == None:
                break
            else:
                cur = cur.parent
    
    def getPi(self, node):
        c_visit = [0.0 if (node.cidx[i] == -1) else node.children[node.cidx[i]].N for i in range(N*N)]
        print(c_visit)
        #if(node.children):
        #    assert(int(sum(c_visit)-len(node.children)*10) == node.N-10 )
        c_p = [x/node.N for x in c_visit]
        return c_p

In [69]:
pp = PUCT_Agent()
pr = Random_Agent()
pr2 = Random_Agent()
pp2 = PUCT_Agent()
pp3 = PUCT_Agent()
ph = Human_Agent()
g = Game(pp, pp2)
#g2 = Game(pr, pr2)
g2 = Game(pr, pr2)

In [64]:
#raise NameError('HiThere')

In [65]:
play2 = g.play()

[59, 59, 59, 59, 59, 59, 59, 446, 59]
[15, 17, 13, 13, 817, 13, 15, 0.0, 13]
[10, 10, 11, 10, 0.0, 10, 404, 0.0, 459]
[5, 5, 5, 5, 0.0, 5, 887, 0.0, 0.0]
[5, 5, 890, 5, 0.0, 5, 0.0, 0.0, 0.0]
[5, 5, 0.0, 5, 0.0, 893, 0.0, 0.0, 0.0]
[357, 357, 0.0, 192, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 897, 0.0, 7, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 902, 0.0, 0.0, 0.0, 0.0, 0.0]


In [66]:
print(play)
print(play2)


(0, [[1, 4], [-1, 8], [1, 7], [-1, 1], [1, 2], [-1, 6], [1, 3], [-1, 5], [1, 0]])
(0, [[1, 7], [-1, 4], [1, 8], [-1, 6], [1, 2], [-1, 5], [1, 0], [-1, 1], [1, 3]])


In [67]:
Game.display(play2)

[['.' '.' '.']
 ['.' '.' '.']
 ['.' 'o' '.']]
[['.' '.' '.']
 ['.' 'x' '.']
 ['.' 'o' '.']]
[['.' '.' '.']
 ['.' 'x' '.']
 ['.' 'o' 'o']]
[['.' '.' '.']
 ['.' 'x' '.']
 ['x' 'o' 'o']]
[['.' '.' 'o']
 ['.' 'x' '.']
 ['x' 'o' 'o']]
[['.' '.' 'o']
 ['.' 'x' 'x']
 ['x' 'o' 'o']]
[['o' '.' 'o']
 ['.' 'x' 'x']
 ['x' 'o' 'o']]
[['o' 'x' 'o']
 ['.' 'x' 'x']
 ['x' 'o' 'o']]
[['o' 'x' 'o']
 ['o' 'x' 'x']
 ['x' 'o' 'o']]


In [74]:
Game.display(Game(ph,pr).play())

[['.' '.' '.']
 ['.' '.' '.']
 ['.' '.' '.']]
input move (x, y) as 3*x+y (0-indexed):4
[['.' '.' '.']
 ['.' 'o' 'x']
 ['.' '.' '.']]
input move (x, y) as 3*x+y (0-indexed):0
[['o' '.' '.']
 ['.' 'o' 'x']
 ['.' '.' 'x']]
input move (x, y) as 3*x+y (0-indexed):8
occupied!!
input move (x, y) as 3*x+y (0-indexed):7
[['.' '.' '.']
 ['.' 'o' '.']
 ['.' '.' '.']]
[['.' '.' '.']
 ['.' 'o' 'x']
 ['.' '.' '.']]
[['o' '.' '.']
 ['.' 'o' 'x']
 ['.' '.' '.']]
[['o' '.' '.']
 ['.' 'o' 'x']
 ['.' '.' 'x']]
[['o' '.' '.']
 ['.' 'o' 'x']
 ['.' 'o' 'x']]
[['o' '.' 'x']
 ['.' 'o' 'x']
 ['.' 'o' 'x']]
